In [2]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as BeautifulSoup
import pandas as pd
import yagmail
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# from api_keys import username, password

tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')


In [3]:
browser = Browser('chrome')

In [129]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from io import StringIO

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)

# Set up the webdriver with Chrome
driver = webdriver.Chrome(options=chrome_options)

url = "https://www.nhl.com/scores/htmlreports/20232024/PL020175.HTM"

try:
    # Open the webpage with Selenium
    driver.get(url)

    # Get the HTML content after the dynamic content has loaded
    html_content = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Now you can work with the soup object to extract data from the HTML.
    tables = soup.find_all('table', class_='tablewidth')

    dataframes = [pd.read_html(str(table))[0] for table in tables]

    # Print the extracted DataFrames
    for index, df in enumerate(dataframes, 1):
        print(f"DataFrame {index}:\n{df}\n{'='*50}\n")

finally:
    # Close the webdriver
    driver.quit()

DataFrame 1:
                                                    0   \
0    VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...   
1    VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...   
2                                              VISITOR   
3                                                    2   
4                                                  NaN   
..                                                 ...   
802                                                 60   
803                                                  D   
804                                                 36   
805                                                  G   
806  © Copyright 2023, National Hockey League  2023...   

                                                    1   \
0    VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...   
1    Play By Play  Sunday, November 5, 2023 Attenda...   
2                                                  NaN   
3                                                  NaN   


In [128]:
dataframes[6]

,0,1,2,3,4,5,6,7,8,9,10
0,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,NaN,NaN,NaN
1,VISITOR 2 VEGAS GOLDEN KNIGHTS Game 13 Away Ga...,"Play By Play Sunday, November 5, 2023 Attenda...",HOME 4 ANAHEIM DUCKS Game 11 Home Game 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VISITOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
66,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
dataframes[0][4].value_counts()

num_tables = len(dataframes)

events_df = pd.DataFrame()

f=1
for frame in range(num_tables):
    name = f"table_{str(f)}"
    globals()[name] =  dataframes[frame]
    f+=1
    globals()[name] = globals()[name].loc[globals()[name].iloc[:, 5].notna()]
    globals()[name] = globals()[name].iloc[1:]
    new_headers = globals()[name].iloc[0]
    globals()[name] = globals()[name][1:].set_axis(new_headers, axis=1)
    globals()[name] = globals()[name].iloc[:-1]
    
    events_df = pd.concat([events_df, globals()[name]], axis=0, ignore_index=True)


In [106]:
import re
import numpy as np

for index, row in events_df.iterrows():
    events_df.at[index, 'Corsi'] = row['Description'][:3]

    
events_df = events_df.loc[(events_df['Corsi'] == 'ANA') | (events_df['Corsi'] == 'VGK')]
events_df

19,#,Per,Str,Time: Elapsed Game,Event,Description,VGK On Ice,ANA On Ice,NaN,NaN,NaN,Corsi
1,5,1,EV,0:00 20:00,FAC,VGK won Neu. Zone - VGK #71 KARLSSON vs ANA #1...,71 C 22 R 16 L 3 D 27 D 36 G,14 C 17 C 33 R 5 D 7 D 36 G,NaN,NaN,NaN,VGK
2,6,1,EV,0:27 19:33,TAKE,"VGK TAKEAWAY - #27 THEODORE, Neu. Zone",71 C 22 R 16 L 3 D 27 D 36 G,14 C 17 C 33 R 5 D 7 D 36 G,NaN,NaN,NaN,VGK
3,7,1,EV,1:13 18:47,SHOT,"VGK ONGOAL - #6 KORCZAK, Wrist , Off. Zone, 30...",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK
4,8,1,EV,1:23 18:37,SHOT,"VGK ONGOAL - #9 EICHEL, Snap , Off. Zone, 35 ft.",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK
5,9,1,EV,1:47 18:13,HIT,"VGK #28 CARRIER HIT ANA #46 LYUBUSHKIN, Off. Zone",20 C 61 R 28 L 17 D 94 D 36 G,11 C 91 C 19 R 34 D 46 D 36 G,NaN,NaN,NaN,VGK
...,...,...,...,...,...,...,...,...,...,...,...,...
252,256,3,EV,18:51 1:09,GOAL,"ANA #39 CARRICK(3), Wrist , Def. Zone, 132 ft....",9 C 20 C 61 R 81 R 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,ANA
253,257,3,EV,18:51 1:09,FAC,ANA won Neu. Zone - VGK #22 AMADIO vs ANA #14 ...,71 C 22 R 16 L 6 D 17 D 36 G,14 C 17 C 33 R 7 D 60 D 36 G,NaN,NaN,NaN,ANA
255,259,3,EV,19:09 0:51,FAC,VGK won Neu. Zone - VGK #71 KARLSSON vs ANA #1...,71 C 22 R 16 L 6 D 17 D 36 G,14 C 17 C 33 R 7 D 60 D 36 G,NaN,NaN,NaN,VGK
256,260,3,EV,19:22 0:38,SHOT,"VGK ONGOAL - #16 DOROFEYEV, Slap , Off. Zone, ...",71 C 22 R 16 L 6 D 17 D 36 G,14 C 17 C 33 R 7 D 60 D 36 G,NaN,NaN,NaN,VGK


In [107]:
valid_types = ['SHOT', 'BLOCK', 'MISS', 'GOAL']
events_df = events_df[events_df['Event'].isin(valid_types)]
events_df

19,#,Per,Str,Time: Elapsed Game,Event,Description,VGK On Ice,ANA On Ice,NaN,NaN,NaN,Corsi
3,7,1,EV,1:13 18:47,SHOT,"VGK ONGOAL - #6 KORCZAK, Wrist , Off. Zone, 30...",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK
4,8,1,EV,1:23 18:37,SHOT,"VGK ONGOAL - #9 EICHEL, Snap , Off. Zone, 35 ft.",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK
7,11,1,EV,3:33 16:27,MISS,"VGK #7 PIETRANGELO, Snap, High and Wide Right,...",71 C 22 R 16 L 6 D 7 D 36 G,14 C 17 C 33 R 34 D 46 D 36 G,NaN,NaN,NaN,VGK
9,13,1,EV,4:42 15:18,BLOCK,ANA #77 VATRANO OPPONENT-BLOCKED BY VGK #94 PA...,9 C 49 C 81 R 17 D 94 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,ANA
11,15,1,EV,5:17 14:43,MISS,"VGK #28 CARRIER, Backhand, Wide Left, Off. Zon...",20 C 61 R 28 L 3 D 27 D 36 G,11 C 91 C 19 R 5 D 7 D 36 G,NaN,NaN,NaN,VGK
...,...,...,...,...,...,...,...,...,...,...,...,...
243,247,3,EV,16:56 3:04,MISS,"ANA #7 GUDAS, Wrist, Wide Right, Off. Zone, 61...",71 C 22 R 16 L 17 D 94 D 36 G,39 C 19 R 20 R 5 D 7 D 36 G,NaN,NaN,NaN,ANA
248,252,3,EV,18:22 1:38,BLOCK,VGK #16 DOROFEYEV OPPONENT-BLOCKED BY ANA #46 ...,9 C 71 C 22 R 16 L 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,VGK
249,253,3,EV,18:28 1:32,MISS,"ANA #39 CARRICK, Slap, Wide Right, Def. Zone, ...",9 C 71 C 22 R 16 L 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,ANA
252,256,3,EV,18:51 1:09,GOAL,"ANA #39 CARRICK(3), Wrist , Def. Zone, 132 ft....",9 C 20 C 61 R 81 R 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,ANA


In [108]:
# for index, row in events_df.iterrows():
#     vgk_onice = re.findall(r'\b\d+\b', row['VGK On Ice'])
#     ana_onice = re.findall(r'\b\d+\b', row['ANA On Ice'])
#     print(vgk_onice)
#     vgk_onice = [int(num) for num in vgk_onice]
#     ana_onice = [int(num) for num in ana_onice]
#     vgk_onice = np.array(vgk_onice + [np.nan] * (6 - len(vgk_onice)))
#     ana_onice = np.array(ana_onice + [np.nan] * (6 - len(ana_onice)))
#     events_df.at[index, 'vgk_onice'] = vgk_onice
#     events_df.at[index, 'ana_onice'] = ana_onice

In [109]:
def extract_numbers(player_info):
    return [int(num) for num in player_info.split() if num.isdigit()]

In [110]:
events_df['VGKNumbers'] = events_df['VGK On Ice'].apply(extract_numbers)
events_df['ANANumbers'] = events_df['ANA On Ice'].apply(extract_numbers)
events_df

C:\Users\seana\AppData\Local\Temp\ipykernel_37260\386203928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['VGKNumbers'] = events_df['VGK On Ice'].apply(extract_numbers)
C:\Users\seana\AppData\Local\Temp\ipykernel_37260\386203928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['ANANumbers'] = events_df['ANA On Ice'].apply(extract_numbers)


19,#,Per,Str,Time: Elapsed Game,Event,Description,VGK On Ice,ANA On Ice,NaN,NaN,NaN,Corsi,VGKNumbers,ANANumbers
3,7,1,EV,1:13 18:47,SHOT,"VGK ONGOAL - #6 KORCZAK, Wrist , Off. Zone, 30...",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK,"[9, 49, 81, 6, 7, 36]","[16, 23, 77, 4, 60, 36]"
4,8,1,EV,1:23 18:37,SHOT,"VGK ONGOAL - #9 EICHEL, Snap , Off. Zone, 35 ft.",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK,"[9, 49, 81, 6, 7, 36]","[16, 23, 77, 4, 60, 36]"
7,11,1,EV,3:33 16:27,MISS,"VGK #7 PIETRANGELO, Snap, High and Wide Right,...",71 C 22 R 16 L 6 D 7 D 36 G,14 C 17 C 33 R 34 D 46 D 36 G,NaN,NaN,NaN,VGK,"[71, 22, 16, 6, 7, 36]","[14, 17, 33, 34, 46, 36]"
9,13,1,EV,4:42 15:18,BLOCK,ANA #77 VATRANO OPPONENT-BLOCKED BY VGK #94 PA...,9 C 49 C 81 R 17 D 94 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,ANA,"[9, 49, 81, 17, 94, 36]","[16, 23, 77, 4, 60, 36]"
11,15,1,EV,5:17 14:43,MISS,"VGK #28 CARRIER, Backhand, Wide Left, Off. Zon...",20 C 61 R 28 L 3 D 27 D 36 G,11 C 91 C 19 R 5 D 7 D 36 G,NaN,NaN,NaN,VGK,"[20, 61, 28, 3, 27, 36]","[11, 91, 19, 5, 7, 36]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,247,3,EV,16:56 3:04,MISS,"ANA #7 GUDAS, Wrist, Wide Right, Off. Zone, 61...",71 C 22 R 16 L 17 D 94 D 36 G,39 C 19 R 20 R 5 D 7 D 36 G,NaN,NaN,NaN,ANA,"[71, 22, 16, 17, 94, 36]","[39, 19, 20, 5, 7, 36]"
248,252,3,EV,18:22 1:38,BLOCK,VGK #16 DOROFEYEV OPPONENT-BLOCKED BY ANA #46 ...,9 C 71 C 22 R 16 L 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,VGK,"[9, 71, 22, 16, 7, 27]","[39, 19, 20, 4, 46, 36]"
249,253,3,EV,18:28 1:32,MISS,"ANA #39 CARRICK, Slap, Wide Right, Def. Zone, ...",9 C 71 C 22 R 16 L 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,ANA,"[9, 71, 22, 16, 7, 27]","[39, 19, 20, 4, 46, 36]"
252,256,3,EV,18:51 1:09,GOAL,"ANA #39 CARRICK(3), Wrist , Def. Zone, 132 ft....",9 C 20 C 61 R 81 R 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,ANA,"[9, 20, 61, 81, 7, 27]","[39, 19, 20, 4, 46, 36]"


In [112]:
import requests

playURL = 'https://api-web.nhle.com/v1/gamecenter/' + str(2023020175) + '/play-by-play'
test = requests.get(playURL).json()
test['rosterSpots']

jerseys = pd.DataFrame()

for spot in test['rosterSpots']:
    teamId = spot.get('teamId')
    sweaterNumber = spot.get('sweaterNumber')
    playerId = spot.get('playerId')
    
    temp = pd.DataFrame([{'teamId': teamId, 'sweaterNumber': sweaterNumber, 'playerId': playerId}])
    
    jerseys = pd.concat([jerseys, temp], ignore_index=True)

jerseys

,teamId,sweaterNumber,playerId
0,24,17,8473986
1,54,7,8474565
2,24,14,8474641
3,24,33,8475164
4,54,3,8475188
5,24,7,8475462
6,24,4,8475764
7,24,39,8475842
8,54,61,8475913
9,24,36,8476434


In [114]:
events_df['contains_9'] = events_df['VGKNumbers'].apply(lambda x: 9 in x)
events_df

C:\Users\seana\AppData\Local\Temp\ipykernel_37260\2758319929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['contains_9'] = events_df['VGKNumbers'].apply(lambda x: 9 in x)


19,#,Per,Str,Time: Elapsed Game,Event,Description,VGK On Ice,ANA On Ice,NaN,NaN,NaN,Corsi,VGKNumbers,ANANumbers,contains_9
3,7,1,EV,1:13 18:47,SHOT,"VGK ONGOAL - #6 KORCZAK, Wrist , Off. Zone, 30...",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK,"[9, 49, 81, 6, 7, 36]","[16, 23, 77, 4, 60, 36]",True
4,8,1,EV,1:23 18:37,SHOT,"VGK ONGOAL - #9 EICHEL, Snap , Off. Zone, 35 ft.",9 C 49 C 81 R 6 D 7 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,VGK,"[9, 49, 81, 6, 7, 36]","[16, 23, 77, 4, 60, 36]",True
7,11,1,EV,3:33 16:27,MISS,"VGK #7 PIETRANGELO, Snap, High and Wide Right,...",71 C 22 R 16 L 6 D 7 D 36 G,14 C 17 C 33 R 34 D 46 D 36 G,NaN,NaN,NaN,VGK,"[71, 22, 16, 6, 7, 36]","[14, 17, 33, 34, 46, 36]",False
9,13,1,EV,4:42 15:18,BLOCK,ANA #77 VATRANO OPPONENT-BLOCKED BY VGK #94 PA...,9 C 49 C 81 R 17 D 94 D 36 G,16 C 23 C 77 R 4 D 60 D 36 G,NaN,NaN,NaN,ANA,"[9, 49, 81, 17, 94, 36]","[16, 23, 77, 4, 60, 36]",True
11,15,1,EV,5:17 14:43,MISS,"VGK #28 CARRIER, Backhand, Wide Left, Off. Zon...",20 C 61 R 28 L 3 D 27 D 36 G,11 C 91 C 19 R 5 D 7 D 36 G,NaN,NaN,NaN,VGK,"[20, 61, 28, 3, 27, 36]","[11, 91, 19, 5, 7, 36]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,247,3,EV,16:56 3:04,MISS,"ANA #7 GUDAS, Wrist, Wide Right, Off. Zone, 61...",71 C 22 R 16 L 17 D 94 D 36 G,39 C 19 R 20 R 5 D 7 D 36 G,NaN,NaN,NaN,ANA,"[71, 22, 16, 17, 94, 36]","[39, 19, 20, 5, 7, 36]",False
248,252,3,EV,18:22 1:38,BLOCK,VGK #16 DOROFEYEV OPPONENT-BLOCKED BY ANA #46 ...,9 C 71 C 22 R 16 L 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,VGK,"[9, 71, 22, 16, 7, 27]","[39, 19, 20, 4, 46, 36]",True
249,253,3,EV,18:28 1:32,MISS,"ANA #39 CARRICK, Slap, Wide Right, Def. Zone, ...",9 C 71 C 22 R 16 L 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,ANA,"[9, 71, 22, 16, 7, 27]","[39, 19, 20, 4, 46, 36]",True
252,256,3,EV,18:51 1:09,GOAL,"ANA #39 CARRICK(3), Wrist , Def. Zone, 132 ft....",9 C 20 C 61 R 81 R 7 D 27 D,39 C 19 R 20 R 4 D 46 D 36 G,NaN,NaN,NaN,ANA,"[9, 20, 61, 81, 7, 27]","[39, 19, 20, 4, 46, 36]",True


In [121]:
file_name = f"data/summary_statsS-2023-11-28.csv"
player_biosS = pd.read_csv(file_name, index_col=False)
file_name = f"data/summary_statsG-2023-11-28.csv"
player_biosG = pd.read_csv(file_name, index_col=False)
for index, row in player_biosG.iterrows():
    player_biosG.at[index, 'tertiaryPosition'] = 'G'
player_bios = pd.concat([player_biosS, player_biosG], axis=0)
player_bios

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,secondaryPosition,tertiaryPosition,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,saves,shutout,creaseShare
0,8476453,N. Kucherov,TBL,R,26961,21,15.0,21.0,18.0,98,...,W,F,10.03,3.58,0,60,214.80,NaN,NaN,NaN
1,8476468,J. Miller,VAN,C,26486,22,13.0,20.0,16.0,50,...,C,F,9.77,3.27,0,60,196.20,NaN,NaN,NaN
2,8480069,C. Makar,COL,D,30559,21,6.0,26.0,14.0,51,...,D,D,8.21,3.32,0,61,202.52,NaN,NaN,NaN
3,8477933,S. Reinhart,FLA,C,26057,21,15.0,12.0,10.0,55,...,C,F,9.10,3.14,0,61,191.54,NaN,NaN,NaN
4,8480800,Q. Hughes,VAN,D,32684,22,8.0,25.0,15.0,68,...,D,D,7.17,2.96,0,60,177.60,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,8481611,P. Kochetkov,CAR,G,22230,7,NaN,NaN,NaN,151,...,NaN,G,0.26,0.23,0,19,4.95,133.0,1.0,30.81
72,8481692,D. Wolf,CGY,G,3585,1,NaN,NaN,NaN,38,...,NaN,G,-1.21,-1.20,0,2,-3.25,34.0,0.0,4.49
73,8482221,D. Levi,BUF,G,29916,9,NaN,NaN,NaN,251,...,NaN,G,-0.60,-0.56,0,22,-13.68,220.0,0.0,37.91
74,8482411,H. Shepard,WSH,G,7200,2,NaN,NaN,NaN,59,...,NaN,G,4.40,4.40,0,7,31.03,54.0,0.0,11.02


In [126]:
ana = jerseys.loc[jerseys['teamId'] == 24]
vgk = jerseys.loc[jerseys['teamId'] == 54]
for player in ana['sweaterNumber'].tolist():
    temp = events_df.copy()
    temp['contains_player'] = events_df['ANANumbers'].apply(lambda x: player in x)
    temp = temp[temp['contains_player'] == True]
    CorsiFor = len(temp.loc[temp['Corsi'] == 'ANA'])
    CorsiAgainst = len(temp.loc[temp['Corsi'] == 'VGK'])
    PID = ana.loc[ana['sweaterNumber'] == player]['playerId'].iloc[0]
    name = player_bios.loc[player_bios['playerId'] == PID]['name'].iloc[0]
    print(f"{name}: For = {CorsiFor}; Against = {CorsiAgainst}")

A. Killorn: For = 15; Against = 18
A. Henrique: For = 16; Against = 12
J. Silfverberg: For = 15; Against = 15
R. Gudas: For = 15; Against = 21
C. Fowler: For = 18; Against = 16
S. Carrick: For = 9; Against = 14
J. Gibson: For = 53; Against = 60
R. Strome: For = 16; Against = 16
R. Johnston: For = 3; Against = 8
F. Vatrano: For = 16; Against = 20
T. Terry: For = 18; Against = 11
U. Vaakanainen: For = 17; Against = 19
L. Dostal: For = 0; Against = 0
I. Lyubushkin: For = 20; Against = 21
B. Leason: For = 7; Against = 14
T. Zegras: For = 14; Against = 12
J. LaCombe: For = 11; Against = 20
M. McTavish: For = 15; Against = 24
P. Mintyukov: For = 20; Against = 24
L. Carlsson: For = 16; Against = 4
